Created on Monday 04 January 2021  

# **POC Crawler API Google quotidien**
**Group 2 - Recherche de nouvelles sources**  
*Projet Inter-Promo 2021 de la formation SID, Université Paul Sabatier, Toulouse*

@authors : Michaël Corbeau, Marianne Manson, Louis Marquez, Nicolas Enjalbert Courrech

L'objectif de notre groupe était d'identifier de nouvelles sources d'information pertinentes et de les scraper. Nous avons développé des crawlers quotidiens spécifiques aux différents sites trouvés par le crawler de nouvelles sources. Avec l'objectif de chercher des articles sur des sources nouvellemment définies comme pertinentes, la solution de faire des crawler spécifiques n'est pas la plus optimale car elle demande une intervention humaine et d'ingéniérie pour faire ces crawler spécifiques. Nous vous présentons ici une preuve de concept d'un crawler quotidien générique basé sur une API Google visant à remplacer la totalité ou au moins une grande partie des crawlers quotidiens spécifiques. 

Nous avons choisi une API google car elle nous permet de récupérer les résultats des recherches, ce qui s'avère assez compliqué sans API (mesures de protection prises par google pour éviter l'utilisation abusive de leur moteur de recherche). Ce crawler permet d'automatiser le processus de recherche de nouveaux articles quotidiens à partir d'une liste de sites présélectionnés et définis comme pertinents dans le cadre de la veille technologique. 

Ces sites-ci ont vocation à être consulté quotidiennement afin de connaître les derniers articles publiés. De manière naïve, l'ensemble des requêtes faite au moteur de recherche aurait le format suivant "site:site-i.com mots-clef_j". En combinant toutes les possibilités, nous obtenons un ensemble d'équation de taille $ K*P$ avec $K$ le nombre de sites et $P$ le nombre de mots clefs. Dans notre cas, nous avons définis dans un premier temps 30 sites à consultés quotidiennements et plus de 15000 mots clefs (résultat du produit cartésien de deux listes de mots clefs répondant par assemblage à la thématique de recherche). Dans ce cas précis cela retourne plus de 450 000 équations de recherche différentes. Ce nombre de requête est bien trop important pour être traité quotidiennement. Ce nombre aura tendance à augmenter au fur et à mesure que le nombre de sites pertinents augmentent.

En faisant quelques essaies, nous nous sommes rendus compte que les sites visés retourne en moyenne pas plus de 10 articles par jour. L'intérêt est de réaliser des requêtes par site et non plus par mots clés, ce qui réduit drastiquement le nombre de requêtes. 

Comme il est recommandé de ne pas effectuer plus d'une requête par minute avec la verison gratuite de l'API, nous avons optimisé le traitement en couplant le crawling et le scraping. Chaque crawl de site est directement suivi du scraping des articles correspondants pendant la durée d'attente d'une minute avant le prochain crawl.

## Estimation des coûts de l'utilisation de l'API scraperAPI

- gratuit : 1000 requêtes / 5 requêtes concurrentes
- 29\\$ par mois = 250 000 requêtes / 10 requêtes concurrentes
- 99\\$ par mois = 1 000 000 requêtes / 25 requêtes concurrentes
- 249\\$ par mois = 3 000 000 requêtes / 50 requêtes concurrentes
- sur demande = requêtes illimités / requêtes concurrentes illimitées

Plus d'informations sur https://www.scraperapi.com/pricing

Pour la preuve de concept, nous utilisons 26 sites et 1 requête par page de résultats retournée. Comme les recherches sont sur les dernières 24h, il y a le plus souvent 1 seule page de résultats, 3 au maximum.


In [1]:
import time
import json
from urllib.parse import urlencode
from requests import get
import pandas as pd
from BigScraper import BigScraper

In [2]:
# API Key (created on Scraper API)
API_KEY = '2daa0fbbc103c5172e706fcbc5845747'
#'fff2df1787bf81bfe98277920f9a9fcb'

In [3]:
def create_google_url(url_site):
    """
    Create the url for the google search of the new ressources 
    posted by the website during the last 24 hours
    
    Parameters:
        url_site : string of the website url
    
    Out:
        google : url of the search
    
    """
    google = "https://www.google.com/search?hl=fr"
    #search on a specific site
    google += "&q=site%3A" + url_site
    #use of the google inner syntax to restrict the request to the last 24 hours
    google += "&as_qdr=d"
    return google

In [4]:
def get_api_url(url):
    """ 
    Creation of the URL that will allow the legal scraping of Google results (use of the API key). 
    This URL is equivalent to a Google search.

    Parameter :
        url : google URL created from the url website (create_google_url)
    
    Out :
        proxy_url : URLs built using the API
    """

    payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr'}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
    return proxy_url

def scraping(url):     
    """ 
    Scraping with scraperapi of the different pages of the google search

    Parameter :
        url : URL built using the API
    
    Out :
        list_src : list of scraping results for the different pages
    """
    headers = {'Accept': 'application/json'}
    response = get(url, headers = headers)
    source = response.text
    list_src = [source]

    dic = json.loads(source)
    next_page = dic['pagination']['nextPageUrl']

    if next_page is not None:
        time.sleep(60)
        list_src.extend(scraping(get_api_url(next_page)))

    
    return list_src

def get_links(result_scrap):
    """ 
    Retrieval of the links from the scraping results of the google search

    Parameter :
        result_scrap : list of scraping results
    
    Out :
        list_links : URL list
    """
    list_links = []
    for page in result_scrap:
        try:
            dico = json.loads(page)
            result = dico['organic_results']

            for i in range(len(result)):
                list_links.append(result[i]['link'])
        except: 
            list_links = []
    return list_links

In [5]:
def get_google_links_list(url):
    """ 
    Given a preselectionned website, returns a list with all the links which 
    will be used to scrap the corresponding articles

    Parameter :
        url : url retrieved by the global crawler 
    
    Out :
        list_links : articles links list
    """

    return get_links(scraping(get_api_url(create_google_url(url))))

def get_dataframe(liste, df):
    """ 
    From a list of articles to scrap and a predefined dataframe, 
    returns a dataframe with all the articles scraped

    Parameter :
        liste : articles links list
        df : an empty df with the named columns
    
    Out :
        df : a df filled with the scraped articles
    """
    
    i = 0
    delay = 0
    bs = BigScraper()
    start_time = time.time() 
    for link in liste:
        try: 
            df.loc[i] = bs.scrap(link)
            print(link + " has been successfully added to the dataframe")
            i += 1
        except:
            print(link + " is not a valid article")
    interval = int(time.time() - start_time)

    #it's recommended to execute 1 request only per minute in the free API version
    if interval < 60:
        delay = 60 - interval
        print(str(delay) + " s before the next request")
        time.sleep(delay)
    return df

In [6]:
'''
list of preselectionned urls from the global scraper
'''

liste_urls = ['https://www.zdnet.fr',
              'https://www.cnil.fr',
              'https://www.fnccr.asso.fr',
              'https://grh-multi.net/fr',
              'https://www.theinnovation.eu',
              'https://www.inserm.fr',
              'https://www.parlonsrh.com',
              'https://www.myrhline.com',
              'https://changethework.com',
              'https://blockchainfrance.net',
              'https://citoyen-ne-s-de-marseille.fr',
              'https://hellofuture.orange.com',
              'https://www.data.gouv.fr',
              'https://www.digitalrecruiters.com',
              'https://www.erudit.org/fr',
              'https://www.lebigdata.fr',
              'https://www.linternaute.fr',
              'https://www.riskinsight-wavestone.com',
              'https://search.sap.com',
              'https://www.cadre-dirigeant-magazine.com',
              'https://www.journaldunet.com',
              'https://www.usine-digitale.fr',
              'https://www.usinenouvelle.com',
              'https://www.lemondeinformatique.fr',
              'https://www.silicon.fr',
              'https://www.lemonde.fr']


In [7]:
def all_articles_to_df(liste_urls):
    """ 
    From a list of websites to scrap and a predefined dataframe, 
    returns a dataframe with all the websites scraped

    Parameter :
        liste_urls : websites links list
        an empty df with the named columns
    
    Out :
        df : a df filled with the scraped websites
    """
    
    df_final = pd.DataFrame(columns=['art_content','art_content_html','art_published_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','art_img','art_auth','art_tag']  )

    for website in liste_urls:
        print(website)
        test = get_google_links_list(website)
        print("total results : "+str(len(test)))
        df = pd.DataFrame(columns=['art_content','art_content_html','art_published_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','art_img','art_auth','art_tag']  )

        df = get_dataframe(test, df)
        df_final = df_final.append(df)
    
    return df_final

In [ ]:
df = all_articles_to_df(liste_urls)

https://www.zdnet.fr
total results : 10
https://www.zdnet.fr/actualites/adieu-token-place-au-cyberjeton-39916309.htm has been successfully added to the dataframe
https://www.zdnet.fr/actualites/secnumcloud-tout-comprendre-en-cinq-points-39916267.htm has been successfully added to the dataframe
https://www.zdnet.fr/actualites/le-bitcoin-rebondit-christine-lagarde-tres-mefiante-39916263.htm has been successfully added to the dataframe
https://www.zdnet.fr/actualites/xiaomi-rejoint-la-liste-noire-de-l-administration-trump-39916283.htm has been successfully added to the dataframe
https://www.zdnet.fr/actualites/ledger-la-faille-de-juillet-etait-plus-serieuse-qu-annoncee-39916301.htm has been successfully added to the dataframe
https://www.zdnet.fr/actualites/testeurs-pros-asus-business-une-solution-de-mobilite-qui-tient-ses-promesses-39915699.htm has been successfully added to the dataframe
https://www.zdnet.fr/actualites/google-a-elabore-un-nouveau-programme-de-formation-sur-le-cloud-comp

In [ ]:
df

# Conclusion :

Dans notre exemple, avec 26 sites, le processus total dure environ 45 minutes et nous retourne un dataframe avec 233 articles scrapés. Sur les 26 sites, une partie ont retourné un résultat nul (pas d'articles dans les dernières 24 heures) cf. pistes d'amélioration. Notre solution fonctionne avec la version gratuite de l'API google. 

Cette solution apporte certains avantage non résolu par des crawlers spécifiques. Ce déroulement fonctionne effectivement de la même façon pour tous les sites référencés par google et il n'est donc pas nécessaire d'avoir une intervention humaine lorsque de nouveaux sites web sont définis comme pertinents. De plus, le moteur de recherche Google permet de faire une sélection par date lors de cette recherche ce qui permet d'éviter de faire la sélection en post-scrapping. 

Néanmoins, implémenter de cette façon, la recherche ne prend pas en compte la sélection de l'article de façon thématique. L'article sera quand même défini comme répondant à la thématique innovation et gamme de gestion après le traitement du groupe de classification (groupe 5) mais une partie des articles retournés ne parleront peut-être pas de la thématique voulue. Une solution est de faire une recherche de mots clefs directement sur le contenu de l'article une fois qu'il est scrapé. 

# Pistes d'amélioration :

- vérifier si une page contient la balise "meta content = article" afin de détecter si la page à scraper est un article. Cela ne marche pas dans tous les cas, mais cela permettrait d'éviter l'appel du BigScraper pour traiter des pages non pertinentes => gain de temps, augmentation de la fiabilité
- conserver les liens d'articles rejetés pour vérifier qu'ils ne sont effectivement pas scrapables
- associer une requête à un mot-clé d'un lexique pour augmenter la pertinence des résultats des requêtes google
- calculer les fréquences de mises à jour des sites. Beaucoup de sites ne sont pas mis à jour quotidiennement et peuvent donc être scrapés à des intervalles de temps plus espacés ce qui permettrait de diminuer le nombre de requêtes quotidiennes ainsi que le temps de traitement
- pour les sites qui sont souvent mis à jour avec de nombreuses pages, affiner l'url source par dossiers et sous-dossiers afin de réduire le temps de traitement
- la durée de traitement peut être réduite en utilisant des requêtes concurrentes
- on peut augmenter la fiabilité en effectuant des recherches par mot-clé une fois que les articles ont été scrapés

Code pour détecter si la page à scraper est un article :
(Cette fonction n'est pour le moment pas implémentée dans le scraping des derniers articles)

In [ ]:
def sort_articles(url_list):
    """ 
    Sorting of the articles in the url list (given that the article as the tag "<meta property="og:type" content="article">")

    Parameter :
        url_list : URLs of the result of the google search
    
    Out :
        list_links : list with only the links of pages identified as articles
    """
    list_links = []
    for i in range(len(url_list)):
            url = url_list[i]
            req = get(url)
            html_soup = BeautifulSoup(req.text, 'html.parser')
            meta = html_soup.find('meta',{'property':'og:type'})
            if meta is not None:
                og_type = meta['content']
                if og_type == "article":
                    list_links.append(url)
    return list_links